# Notation and formulations for the PRP


 - N -> represents the set of the plant and the customers indexed by i = {0; … ; n}
 - A = {(i,j): i,j ∈ N, i ≠ j} is the set of arcs
 - T = {1,...,l}
 - K = {1,...,m} 

### Parameters:
 - u -> unit production cost;
 - f -> fixed production setup cost;
 - hi -> unit inventory holding cost at node i;
 - cij -> transportation cost from node i to node j;
 - dit -> demand at customer i in period t;
 - C -> production capacity;
 - Q -> vehicle capacity;
 - Li -> maximum or target inventory level at node i; e
 - Ii0 -> initial inventory available at node i.

### Decision variables:

 - pt production quantity in period t;
 - Iit inventory at node i at the end of period t;
 - yt equal to 1 if there is production at the plant in period t, 0 otherwise;
 - z0t the number of vehicles leaving the plant in period t;
 - zit equal to 1 if customer i is visited in period t, 0 otherwise, ∀ i ∈ Nc;
 - xijt if a vehicle travels directly from node i to node j in period t, 0 otherwise;
 - qit quantity delivered to customer i in period t;
 - wit load of a vehicle before making a delivery to customer i in period t.

In [13]:
#################################################################################################
# ProductionRoutingProblemPRP1
# Copyright 2024 Mateus Chacon

# Este programa é um software livre, você pode redistribuí-lo e/ou modificá-lo
# sob os termos da Licença Pública Geral GNU como publicada pela Fundação do Software Livre (FSF),
# na versão 3 da Licença, ou (a seu critério) qualquer versão posterior.

# Este programa é distribuído na esperança de que possa ser útil, mas SEM NENHUMA GARANTIA,
# e sem uma garantia implícita de ADEQUAÇÃO a qualquer MERCADO ou APLICAÇÃO EM PARTICULAR.

# Veja a Licença Pública Geral GNU para mais detalhes
#################################################################################################

import gurobipy as gp
from gurobipy import GRB

class Data:
    def __init__(self,costPeriodT, costCustomerI, costCustomerJ):
        self.u
        self.f
        self.h
        self.c
        self.d
        self.C
        self.Q
        self.L
        self.I0
        self.m
        self.CONST_PERIOD_T=costPeriodT
        self.CONST_CUSTOMER_I=costCustomerI
        self.CONST_CUSTOMER_J=costCustomerJ

    def setUnitProductionCost(self,u):
        self.u = u
    def setFixedProductionCost(self,f):
        self.f = f
    def setUnitInventoryHoldingCost(self,h):
        self.h = h
    def setTransportationCost(self,c):
        self.c = c
    def setDemandCustomer(self,d):
        self.d = d
    def setProductionCapacity(self,C):
        self.C = C
    def setVehicleCapactiy(self,Q):
        self.Q = Q
    def setMaximumTargetInvetoryLevel(self,L):
        self.L = L
    def setInitalInventory(self,I0):
        self.I0 = I0
    def setMaxTrucks(self,m):
        self.m = m

class ProductionRoutingProblemPRP1:
    def __init__(self, data = Data):
        self.model = gp.Model("Production_Routing_Problem")
        self.CONST_PERIOD_T=data.CONST_PERIOD_T
        self.CONST_CUSTOMER_I=data.CONST_CUSTOMER_I
        self.CONST_CUSTOMER_J=data.CONST_CUSTOMER_J
        self.data = data
        self.p = {}
        self.I = {}
        self.y = {}
        self.z = {}
        self.x = {}
        self.q = {}
        self.w = {}
        self.M = []
        self.MT = []

    def createDecisionVariables(self):
        for t in range(self.CONST_PERIOD_T):
            self.p[t] = self.model.addVar(vtype=GRB.INTEGER, name=f"p[{t}]")
            self.y[t] = self.model.addVar(vtype=GRB.BINARY, name=f"y[{t}]") 
           
        for i in range(self.CONST_CUSTOMER_I):
            for j in range(self.CONST_CUSTOMER_J):
                for t in range(self.CONST_PERIOD_T):
                    self.x[i,j,t] = self.model.addVar(vtype=GRB.BINARY, name=f"x[{i},{j},{t}]") 
        
        for i in range(self.CONST_CUSTOMER_I):
            for t in range(self.CONST_PERIOD_T):
                self.I[i,t] = self.model.addVar(vtype=GRB.INTEGER, name=f"I[{i},{t}]") 
                self.w[i,t] = self.model.addVar(vtype=GRB.INTEGER, name=f"w[{i},{t}]") 
                self.q[i,t] = self.model.addVar(vtype=GRB.INTEGER, name=f"q[{i},{t}]")

                if(i==0):
                    self.z[0,t] = self.model.addVar(vtype=GRB.INTEGER, name=f"z[{i},{t}]") 
                else:
                    self.z[i,t] = self.model.addVar(vtype=GRB.BINARY, name=f"z[{i},{t}]") 
    
    def crateObjectiveFunction(self):
        objExpr = gp.LinExpr()
        
        for t in range(self.CONST_PERIOD_T):
            objExpr_1 = gp.LinExpr()
            for i in range(self.CONST_CUSTOMER_I):
                objExpr_1+= self.data.h[i] + self.I[i,t]

            objExpr_2 = gp.LinExpr()
            for i in range(self.CONST_CUSTOMER_I):
                for j in range(self.CONST_CUSTOMER_J):
                   objExpr_2+= self.data.c[i][j]*self.x[i,j,t] 
                   
            objExpr += self.data.u * self.p[t] + self.data.f * self.y[t] + objExpr_1 + objExpr_2

        self.model.setObjective(objExpr, GRB.MINIMIZE)


    def createRestrictionBalanceStockFlowFactoryAndCustomers(self):
        for i in range(self.CONST_CUSTOMER_I):
            self.model.Add(self.I[i, 0] == self.data.I0, name=f"rest_1_{i}")
        
        for t in range(1,self.CONST_PERIOD_T):
            r2 = 0
            for i in range(self.CONST_CUSTOMER_I):
                r2+=self.q[i,t] + self.I[0,t]
                self.model.Add(  self.I[i, t-1] + self.q[i,t] == self.data.d[i][t]+self.I[i,t], name=f"rest_3_{t}_{i}")
            self.model.Add(  self.I[0, t-1] + self.p[t] == r2, name=f"rest_2_{t}_{i}")

    def createRestrictionProductionCapactiy(self):
        for t in range(self.CONST_PERIOD_T): 
            self.model.Add( self.p[t] <= self.M[t]*self.y[t], name=f"rest_4_{t}")

    def createRestrictionLimitMaximumInventory(self):
        for t in range(self.CONST_PERIOD_T): 
            self.model.Add( self.I[0,t] <= self.data.L[0], name=f"rest_5_{t}")
        
        for t in range(1,self.CONST_PERIOD_T): 
            for i in range(self.CONST_CUSTOMER_I):
                self.model.Add( self.I[i,t-1] + self.q[i,t] <= self.data.L[i], name=f"rest_6_{t}")

    def createRestrictionAllowPositiveDeliveryQuantityOnly(self):
        for t in range(1,self.CONST_PERIOD_T): 
            for i in range(self.CONST_CUSTOMER_I):
                self.model.Add( self.q[i,t]<=self.MT[i][t]*self.z[i,t], name=f"rest_7_{t}_{i}")

    def createRestrictionEachCustomerCanBeVistitedByMostOneVehicle(self):
        for t in range(self.CONST_PERIOD_T):
            for i in range(self.CONST_CUSTOMER_I):
                r8=0
                for j in range(self.CONST_CUSTOMER_J):
                    r8+=self.x[i,j,t]
                self.model.Add(r8==self.z[i,t], name=f"rest_8_{t}_{i}")

    def createRestrictionVehicleFlowConservation(self):
        for t in range(self.CONST_PERIOD_T):
            for i in range(self.CONST_CUSTOMER_I):
                r9_1=0
                for j in range(self.CONST_CUSTOMER_J):
                    r8+=self.x[i,j,t]
        
                r9_2=0
                for j in range(self.CONST_CUSTOMER_J):
                    r8+=self.x[j,i,t]

                self.model.Add(r9_1+r9_2 == 2*self.z[i,t], name=f"rest_9_{t}_{i}")

    def createRestrictionLimitNumberTrucks(self):
        for t in range(self.CONST_PERIOD_T):
            self.model.Add(self.z[0,t]<self.data.m, name=f"rest_10_{t}")

    def createRestrictionVehicleLoadingAndSubTourElimination(self):  
        for i in range(self.CONST_CUSTOMER_I):
            for j in range(self.CONST_CUSTOMER_J):
                for t in range(self.CONST_PERIOD_T):
                    self.model.Add(self.w[i,t] - self.w[j,t] >= self.q[i,t] - self.MT[i][j]*(1 - self.x[i,j,t]), name=f"rest_11_{t}")

        for t in range(self.CONST_PERIOD_T):
            for i in range(self.CONST_CUSTOMER_I):
                self.model.Add(self.w[i,t] <= self.data.Q*self.z[i,t], name=f"rest_11_{t}")


    def solve(self):
        self.createDecisionVariables()
        self.crateObjectiveFunction()
        self.createRestrictionBalanceStockFlowFactoryAndCustomers()
        self.createRestrictionProductionCapactiy()
        self.createRestrictionLimitMaximumInventory()
        self.createRestrictionAllowPositiveDeliveryQuantityOnly()
        self.createRestrictionEachCustomerCanBeVistitedByMostOneVehicle()
        self.createRestrictionVehicleFlowConservation()
        self.createRestrictionLimitNumberTrucks()
        self.createRestrictionVehicleLoadingAndSubTourElimination()




